In [3]:
from pyspark.sql import SparkSession

# Создание сессии Spark
spark = SparkSession.builder \
    .appName("RDD_Analysis") \
    .getOrCreate()

# Доступ к Spark Context
sc = spark.sparkContext

In [4]:
# Считывание файла
warandpeace = sc.textFile("warandsociety.txt")

In [5]:
# Вывод количества строк в файле
print(warandpeace.count())

12851


In [6]:
# Попытка считать несуществующий файл
nilFile = sc.textFile("nil")
print(nilFile.count())  # Должна появиться ошибка, так как файла "nil" не существует

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/nil
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:210)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.api.python.PythonRDD.getPartitions(P

In [7]:
# Считывание 10 первых строк
print(warandpeace.take(10))

['Лев Николаевич Толстой', 'Война и мир. Книга 1', '', 'Война и мир – 1', '', ' ', ' http://www.lib.ru', '', 'Аннотация ', '']


In [8]:
# Информация о количестве разделов
print(warandpeace.getNumPartitions())

2


In [10]:
# Просмотр ip адреса
import socket

rdd = sc.parallelize([1, 2, 3]).map(lambda x: (x, socket.gethostname()))
print(rdd.collect())

[(1, '470a189ebd0d'), (2, '470a189ebd0d'), (3, '470a189ebd0d')]


In [11]:
# Стройки со словом "война"
linesWithWar = warandpeace.filter(lambda x: "война" in x)
print(linesWithWar.first())

– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой верный раб, comme vous dites. [Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист) – я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите.] Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, [Я вижу, что я вас пугаю,] садитесь и рассказывайте.


In [13]:
# Кэширование
linesWithWar.cache()
print(linesWithWar.count())  # Первый запуск
print(linesWithWar.count())  # Второй запуск (должен быть быстрее)

54
54


In [14]:
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
# Сохранение результата в файл
wordCounts.saveAsTextFile("warandpeace_histogram.txt")

In [19]:
# Выведет на каждой строке по букве
import re
p = re.compile('\w+')
letters = p.findall("a b c")
[print(l) for l in letters]

a
b
c


[None, None, None]

# Операции с множествами

In [20]:
# Инициализируем множества
a = sc.parallelize([1,2,3,4])
b = sc.parallelize([3,4,6,7])

In [23]:
# Объединение
print(a.union(b).collect())  # Дублирует числа
print(a.union(b).distinct().collect())  # Убирает дубликаты

[1, 2, 3, 4, 3, 4, 6, 7]
[4, 1, 2, 6, 3, 7]


In [24]:
# Пересечение
print(a.intersection(b).collect())

[4, 3]


In [25]:
# Разность
print(a.subtract(b).collect())

[1, 2]


In [26]:
# Широковещательные переменные
broadcastVar = sc.broadcast([1,2,3])
print(broadcastVar.value) # Для получения значения

[1, 2, 3]


In [27]:
# Аккумулирующие переменные
accum = sc.accumulator(0)
sc.parallelize([1,2,3,4]).foreach(lambda x: accum.add(x))
print(accum.value)  # Должно вывести 10

10


In [28]:
# Создание пары ключ-значение из двух букв
pair = ('a', 'b')

In [29]:
# Для доступа к первому значению
pair[0]

'a'

In [30]:
# Для доступа к второму значению
pair[1]

'b'

# Топ-10 популярных номеров такси

In [31]:
# Загрузка данных
taxi = sc.textFile("nyctaxi.csv")

In [32]:
# Вывод пяти первых строк
for t in taxi.take(5):
    print(t)

"_id","_rev","dropoff_datetime","dropoff_latitude","dropoff_longitude","hack_license","medallion","passenger_count","pickup_datetime","pickup_latitude","pickup_longitude","rate_code","store_and_fwd_flag","trip_distance","trip_time_in_secs","vendor_id"
"29b3f4a30dea6688d4c289c9672cb996","1-ddfdec8050c7ef4dc694eeeda6c4625e","2013-01-11 22:03:00",+4.07033460000000E+001,-7.40144200000000E+001,"A93D1F7F8998FFB75EEF477EB6077516","68BC16A99E915E44ADA7E639B4DD5F59",2,"2013-01-11 21:48:00",+4.06760670000000E+001,-7.39810790000000E+001,1,,+4.08000000000000E+000,900,"VTS"
"2a80cfaa425dcec0861e02ae44354500","1-b72234b58a7b0018a1ec5d2ea0797e32","2013-01-11 04:28:00",+4.08190960000000E+001,-7.39467470000000E+001,"64CE1B03FDE343BB8DFB512123A525A4","60150AA39B2F654ED6F0C3AF8174A48A",1,"2013-01-11 04:07:00",+4.07280540000000E+001,-7.40020370000000E+001,1,,+8.53000000000000E+000,1260,"VTS"
"29b3f4a30dea6688d4c289c96758d87e","1-387ec30eac5abda89d2abefdf947b2c1","2013-01-11 22:02:00",+4.07277180000000E+00

In [34]:
# Фильтр заголовка
import itertools
taxi.mapPartitionsWithIndex(lambda idx, it:  itertools.islice(it,1,None) if (idx==0) else it  )

PythonRDD[73] at RDD at PythonRDD.scala:53

In [35]:
taxiParse = taxi.map(lambda line: line.split(","))

In [36]:
# Преобразуем массив строк в массив пар ключ-значение, где ключом будет служить номер такси (6 колонка), а значением единица
taxiMedKey = taxiParse.map(lambda row: (row[6], 1))

In [37]:
# Количество поездок каждого номера такси
taxiMedCounts = taxiMedKey.reduceByKey(lambda v1, v2: v1+v2)

In [38]:
# Вывод отсортированных результатов
top10 = taxiMedCounts.map(lambda x: x[::-1]).top(10)
for x in top10:
    print(x[::-1])

('"AB44AD9A03B7CFAF3925103BDCC0AF23"', 44)
('"71CACFBADF9568AAE88A843DB511D172"', 41)
('"6483B9BFCB216EC88986EA3AB13064E7"', 41)
('"4C73459B430339981D78795300433438"', 41)
('"67E71D24AF704D814A0A825005ADA72E"', 40)
('"02E5A4136FD0A775A023A005A4EABC62"', 40)
('"9DFBCD218E7116F34C044F0680A0FB8A"', 39)
('"8DEB70907D00AA1D7FF5E2683240549B"', 39)
('"7989C2AB3F345F4AB54D3CF1E0480D67"', 39)
('"6C9F67DF658DC5636F9E7752F203F70A"', 39)


In [48]:
# Преобразуем исходные данные в одну цепочку
taxiCounts = taxi.map(lambda line: line.split(",")).map(lambda row: (row[6],1)).reduceByKey(lambda a,b: a + b)

In [49]:
# Общее количество номеров такси
taxiCounts.cache()

PythonRDD[90] at RDD at PythonRDD.scala:53

In [51]:
# Сравнение времени
import time

# Функция для измерения времени выполнения
def time_it(func):
    start = time.time()  # Фиксируем стартовое время
    result = func()  # Вызываем переданную функцию
    end = time.time()  # Фиксируем конечное время
    print(f"Время выполнения: {end - start:.4f} сек.")
    return result

# Кэшируем RDD перед измерением времени
taxiCounts.cache()

# Первый вызов count() (до кэширования)
print("Первый вызов count() (данные еще не кэшированы):")
time_it(lambda: taxiCounts.count())

# Второй вызов count() (должен быть быстрее)
print("Второй вызов count() (должен быть быстрее):")
time_it(lambda: taxiCounts.count())

Первый вызов count() (данные еще не кэшированы):
Время выполнения: 2.8720 сек.
Второй вызов count() (должен быть быстрее):
Время выполнения: 0.5264 сек.


13371

# Настройка способа хранения RDD

In [52]:
import pyspark
taxi.persist(storageLevel=pyspark.StorageLevel.MEMORY_ONLY)

nyctaxi.csv MapPartitionsRDD[70] at textFile at NativeMethodAccessorImpl.java:0